In [17]:
import pandas as pd
import numpy as np
from datetime import datetime
regularseasonraw = pd.read_csv('MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
postseasonraw = pd.read_csv('MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')

In [18]:
def add_rate_cols(df):
    
    df['W_possessions'] = .5 * (df['WFGA'] + (.475 * df['WFTA']) - df['WOR'] + df['WTO'])
    df['WeFG'] = (df['WFGM'] + (.5 * df['WFGM3'])) / df['WFGA']
    df['WTSpct'] = df['WScore'] / (2 * (df['WFGA'] + .475 * df['WFTA']))
    df['WFTR'] = df['WFTA'] / df['WFGA']
    df['W3PR'] = df['WFGA3'] / (df['WFGA'] + (.475*df['WFTA']))
    df['WORpct'] = df['WOR'] / (df['WOR'] + df['LDR'])
    df['WDRpct'] = df['WDR'] / (df['WDR'] + df['LOR'])
    df['WREBpct'] = (df['WOR'] + df['WDR']) / (df['WOR'] + df['LDR'] + df['LOR'] + df['WDR'])
    df['WTR'] = (df['WOR'] + df['WDR'])
    df['WATOr'] = df['WAst'] / df['WTO']
    df['WAst_pct'] = df['WAst'] / df['WFGM']
    df['WStl_pct'] = df['WStl'] / (df['LFGA'] + .475*df['LFTA'] + df['LTO'])
    df['WBlk_pct'] = df['WBlk'] / (df['LFGA'])
    df['WTO_ratio'] = df['WTO'] / (df['WFGA'] + .475*df['WFTA'] + df['WTO'])
    df['L_possessions'] = .5 * (df['LFGA'] + (.475 * df['LFTA']) - df['LOR'] + df['LTO'])
    df['LeFG'] = (df['LFGM'] + (.5 * df['LFGM3'])) / df['LFGA']
    df['LTSpct'] = df['LScore'] / (2 * (df['LFGA'] + .475 * df['LFTA']))
    df['LFTR'] = df['LFTA'] / df['LFGA']
    df['L3PR'] = df['LFGA3'] / (df['LFGA'] + (.475*df['LFTA']))
    df['LORpct'] = df['LOR'] / (df['LOR'] + df['WDR'])
    df['LDRpct'] = df['LDR'] / (df['LDR'] + df['WOR'])
    df['LREBpct'] = (df['LOR'] + df['LDR']) / (df['WOR'] + df['LDR'] + df['LOR'] + df['WDR'])
    df['LTR'] = (df['LOR'] + df['LDR'])
    df['LATOr'] = df['LAst'] / df['LTO']
    df['LAst_pct'] = df['LAst'] / df['LFGM']
    df['LStl_pct'] = df['LStl'] / (df['WFGA'] + .475*df['WFTA'] + df['WTO'])
    df['LBlk_pct'] = df['LBlk'] / (df['WFGA'])
    df['LTO_ratio'] = df['LTO'] / (df['LFGA'] + .475*df['LFTA'] + df['LTO'])
    
    """df = df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'W_possessions', 'WeFG', 'WTSpct', 'WFTR', 'W3PR', 'WORpct', 'WDRpct',
       'WREBpct', 'WTR', 'WATOr', 'WAst_pct', 'WStl_pct', 'WBlk_pct',
       'WTO_ratio', 'L_possessions', 'LeFG', 'LTSpct', 'LFTR', 'L3PR',
       'LORpct', 'LDRpct', 'LREBpct', 'LTR', 'LATOr', 'LAst_pct', 'LStl_pct',
       'LBlk_pct', 'LTO_ratio']]
    """
    
    return df

In [19]:
regularseason = regularseasonraw
postseason = postseasonraw
regularseason.DayNum.value_counts().sort_index()

0        2
1      186
2       41
3       20
4      381
      ... 
128    496
129    914
130    688
131    369
132     83
Name: DayNum, Length: 133, dtype: int64

In [20]:
df = regularseason

In [22]:
def team_regular_season(team_id,Season = all, DayNum = all, average = True):
    
    teamwins = regularseason[(regularseason['WTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)] 
    teamlosses = regularseason[(regularseason['LTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)]
    
    teamwinsoffense = teamwins.filter(like = 'W')    
    teamlossesoffense = teamlosses.filter(like = 'L')

    teamwinsoffense.columns = teamwinsoffense.columns.str.lstrip('W')
    teamlossesoffense.columns = teamlossesoffense.columns.str.lstrip('L')

    teamoffense = pd.concat([teamwinsoffense, teamlossesoffense])

    teamwinsopponents = teamwins.filter(like = 'L')
    teamlossesopponents = teamlosses.filter(like = 'W')

    teamwinsopponents.columns = teamwinsopponents.columns.str.lstrip('L')
    teamlossesopponents.columns = teamlossesopponents.columns.str.lstrip('W')

    teamopponents = pd.concat([teamwinsopponents, teamlossesopponents])
    
    teamtotal = pd.concat([teamoffense, teamopponents.add_prefix('Opp_')], axis = 1)
    
    ###cols_to_drop = ['Def_TeamID', 'Def_Score',]
    
    if average:
        teamtotal = teamtotal.groupby('TeamID').mean().reset_index()
        
    return teamtotal

In [23]:
marquette = team_regular_season(1266, 2020, 132, True)
marquette

,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,...,Opp_FGA3,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF
0,1266,77.766667,25.3,58.866667,10.033333,26.233333,17.133333,23.066667,10.7,29.266667,...,21.366667,14.2,21.366667,10.2,24.833333,13.166667,10.966667,6.766667,3.066667,19.466667


In [24]:
def regular_season_game(team1_id, team2_id, Year = 2020, DayNum = 132):
    team1 = team_regular_season(team1_id,Year,DayNum)
    team2 = team_regular_season(team2_id,Year,DayNum)
    game = pd.concat([team1, team2], axis = 1)
    
    return game

In [25]:
schedule2020 = regularseasonraw[(regularseasonraw['Season'] == 2020) & (regularseasonraw['DayNum'] > 12)].loc[:,['WTeamID', 'LTeamID', 'DayNum']]
schedule2020

,WTeamID,LTeamID,DayNum
88037,1112,1308,13
88038,1113,1351,13
88039,1114,1146,13
88040,1117,1225,13
88041,1123,1232,13
...,...,...,...
92827,1389,1233,128
92828,1393,1314,128
92829,1402,1111,128
92830,1412,1427,128


In [26]:
schedule = list(zip(schedule2020.WTeamID,schedule2020.LTeamID,schedule2020.DayNum))

In [27]:
games2020 = pd.DataFrame()
for i in range(len(schedule)):
    games2020 = games2020.append(regular_season_game(schedule[i][0], schedule[i][1], DayNum = schedule[i][2]))

In [28]:
games2020

,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,...,Opp_FGA3,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF
0,1112,89.333333,32.333333,61.333333,8.000000,19.000000,16.666667,23.333333,10.000000,29.666667,...,12.500000,14.000000,18.500000,6.500000,21.500000,7.500000,14.000000,7.500000,2.000000,25.500000
0,1113,80.500000,27.000000,65.000000,7.500000,23.500000,19.000000,25.500000,9.000000,32.000000,...,34.000000,13.000000,18.500000,8.500000,25.000000,11.500000,12.500000,8.500000,1.500000,21.000000
0,1114,68.500000,27.000000,52.500000,4.500000,16.500000,10.000000,13.500000,9.000000,21.000000,...,24.333333,11.333333,16.000000,13.000000,37.666667,20.666667,15.000000,9.000000,5.000000,19.000000
0,1117,64.666667,24.000000,51.333333,5.000000,14.666667,11.666667,16.000000,8.000000,23.333333,...,24.333333,9.333333,11.333333,8.000000,23.666667,15.000000,12.666667,5.666667,2.333333,16.666667
0,1123,71.000000,28.500000,66.500000,8.000000,25.000000,6.000000,13.500000,14.000000,27.000000,...,22.000000,20.500000,27.500000,10.500000,27.500000,17.000000,12.500000,4.000000,4.000000,19.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1389,66.689655,22.724138,55.344828,6.206897,17.827586,15.034483,20.896552,12.344828,24.896552,...,22.928571,12.714286,19.821429,11.571429,23.678571,14.821429,12.928571,6.857143,2.785714,17.357143
0,1393,74.096774,25.322581,58.258065,8.129032,24.709677,15.322581,20.580645,10.516129,25.193548,...,25.093750,13.187500,17.343750,8.937500,25.218750,13.750000,11.562500,6.812500,4.218750,18.750000
0,1402,70.137931,24.931034,56.862069,5.172414,16.310345,15.103448,20.137931,10.103448,24.103448,...,22.533333,11.133333,15.533333,9.733333,25.833333,13.566667,13.933333,6.933333,2.800000,19.400000
0,1412,66.310345,24.034483,53.517241,4.896552,15.827586,13.344828,18.724138,10.310345,26.413793,...,24.071429,16.250000,22.821429,9.714286,29.428571,13.285714,13.392857,5.535714,3.571429,18.357143
